In [1]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt

#Definir region de interes
frac_lim_inf = 1/2
frac_lim_sup = 4/5
def region_of_interest(image):
    height = image.shape[0]
    length = image.shape[1]
    inf = round(frac_lim_inf*height)
    sup = round(frac_lim_sup*height)
    polygons = np.array([
        [(0,inf),(length,inf),(length,sup),(0,sup)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask,polygons,255)
    masked_image = cv2.bitwise_and(image,mask)
    return masked_image

#Funcion de creacion de imagen con lineas
def display_lines(image, lines, lines_yellow, lines_white):
    #Parametros
    line_image = np.zeros_like(image)
    n = 0
    angle = 10
    height = image.shape[0]
    param = []
    new_lines = []
    slope_error = 30
    intercept_error = 30
    #Lineas amarillas
    if lines_yellow is not None:
        color = (255,255,0)
        for line in lines_yellow:
            x1, y1, x2, y2 = line.reshape(4)
            theta = (np.arctan((y2-y1)/(x2-x1)))*180/np.pi;
            if (theta>angle or theta<-angle) and (x1!=x2):
                slope = ((y2-y1)/(x2-x1))
                intercept = int(y1-slope*x1)
                param.append((slope,intercept, color))
                #Lineas amarillas
    if lines_white is not None:
        color = (255,255,255)
        for line in lines_white:
            x1, y1, x2, y2 = line.reshape(4)
            theta = (np.arctan((y2-y1)/(x2-x1)))*180/np.pi;
            if (theta>angle or theta<-angle) and (x1!=x2):
                slope = ((y2-y1)/(x2-x1))
                intercept = int(y1-slope*x1)
                param.append((slope,intercept, color))
    #Otras lineas
    if lines is not None:
        color = (255,0,0)
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            theta = (np.arctan((y2-y1)/(x2-x1)))*180/np.pi;
            if (theta>angle or theta<-angle) and (x1!=x2):
                slope = ((y2-y1)/(x2-x1))
                intercept = int(y1-slope*x1)
                param.append((slope,intercept, color))
    if len(param) is not 0:
        new_lines.append(param[0])
        for line in param:
            slope = line[0]
            intercept = line[1]
            new_lines2 = new_lines
            i=0
            for new_line in new_lines2:
                max_slope = new_line[0]+abs(new_line[0])*slope_error/100
                min_slope = new_line[0]-abs(new_line[0])*slope_error/100
                max_intercept = new_line[1]+abs(new_line[1])*intercept_error/100
                min_intercept = new_line[1]-abs(new_line[1])*intercept_error/100
                if ((slope<max_slope)&(slope>min_slope)&(intercept<max_intercept)&(intercept>min_intercept)):
                    found = True
                    new_slope = (slope+new_line[0])/2
                    new_intercept = (intercept+new_line[1])/2
                    new_line = (new_slope, new_intercept, new_line[2])
                    new_lines[i]=new_line
                    break
                else:
                    found = False
                i=i+1
            if not found:
                new_lines.append(line)
                
    for line in new_lines:
        slope = line[0]
        intercept = line[1]
        y1_new = round(frac_lim_sup*height)
        y2_new = round(frac_lim_inf*height) 
        x1_new = round((y1_new-intercept)/slope)
        x2_new = round((y2_new-intercept)/slope)        
        cv2.line(line_image, (x1_new, y1_new), (x2_new, y2_new), line[2], 4)
    return line_image

#Video
cap = cv2.VideoCapture('video_Trim.mp4')
while True:
    _, image = cap.read()
    
    if (type(image) == type(None)):
        break
        
    #inicio
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    lane_image = np.copy(image)
    gray = cv2.cvtColor(lane_image, cv2.COLOR_RGB2GRAY)
    image_hsv = cv2.cvtColor(lane_image, cv2.COLOR_RGB2HSV)
    
    #lineas blancas
    lowerBound_white = (0,0,130)
    upperBound_white = (255,80,255)
    white = cv2.inRange(image_hsv, lowerBound_white, upperBound_white)
    n=9
    blur = cv2.GaussianBlur(white, (n,n), 0) 
    canny = cv2.Canny(blur, 80, 160)
    cropped_image = region_of_interest(canny);
    lines_white = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 80, np.array([]), minLineLength=30, maxLineGap=3)
    
    #lineas amarillas
    lowerBound_yellow = (8,100,100)
    upperBound_yellow = (34,255,255)
    yellow = cv2.inRange(image_hsv, lowerBound_yellow, upperBound_yellow)
    n=9
    blur = cv2.GaussianBlur(yellow, (n,n), 0) 
    canny = cv2.Canny(blur, 80, 160)
    cropped_image = region_of_interest(canny);
    lines_yellow = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 80, np.array([]), minLineLength=30, maxLineGap=3)
    
    
    #otras lineas
    n=5
    blur = cv2.GaussianBlur(gray, (n,n), 0)  
    canny = cv2.Canny(blur, 80, 160)
    cropped_image = region_of_interest(canny);
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 80, np.array([]), minLineLength=30, maxLineGap=3)

    #Dibujar
    line_image = display_lines(lane_image, lines, lines_yellow, lines_white)
    mixed_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1)
    mixed_image = cv2.cvtColor(mixed_image, cv2.COLOR_BGR2RGB)

    
    cv2.imshow('video', mixed_image)
   
    if cv2.waitKey(33) == 27:
        break

cv2.destroyAllWindows()

KeyboardInterrupt: 